<a href="https://colab.research.google.com/github/PreetiKumari2208/WEB_D-and-ML-projects/blob/main/Twitter_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing kaggle library
!pip install kaggle

In [2]:
# configure the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
# API to fetch the data set from the kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
# extrcting the compressed file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


importing dependencies


In [5]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Data Processing
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [8]:
twitter_data.shape

(1599999, 6)

In [9]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
# Naming the column and reading the dataset again
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',names=column_names)

In [11]:
twitter_data.shape

(1600000, 6)

In [12]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [14]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [15]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [16]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [17]:
def preprocess_text(text):
  text = re.sub(r'http\S+', '', text) # remove hyperlinks
  text=re.sub('RT@[\w]+:','',text)
  text=re.sub('#[\w]+','',text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuations
  text=text.lower()
  text=text.lower()
  words=text.split()
  ps=PorterStemmer()
  words=[ps.stem(word) for word in words if word not in stopwords.words('english')]
  processed_text=' '.join(words)
  return processed_text


In [18]:
twitter_data['stemmed_content']=twitter_data['text'].apply(preprocess_text)

In [19]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot awww that bummer shoulda got david ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset cant updat facebook text might cri resul...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save 50 res...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav im mad cant see


In [20]:
print(twitter_data['stemmed_content'])

0          switchfoot awww that bummer shoulda got david ...
1          upset cant updat facebook text might cri resul...
2          kenichan dive mani time ball manag save 50 res...
3                            whole bodi feel itchi like fire
4                      nationwideclass behav im mad cant see
                                 ...                        
1599995                           woke school best feel ever
1599996           thewdbcom cool hear old walt interview â«
1599997                         readi mojo makeov ask detail
1599998    happi 38th birthday boo alll time tupac amaru ...
1599999              happi thenspcc sparkschar speakinguph4h
Name: stemmed_content, Length: 1600000, dtype: object


In [21]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [22]:
#separating data and labels
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [23]:
print(X)

['switchfoot awww that bummer shoulda got david carr third day'
 'upset cant updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save 50 rest go bound' ...
 'readi mojo makeov ask detail'
 'happi 38th birthday boo alll time tupac amaru shakur'
 'happi thenspcc sparkschar speakinguph4h']


In [24]:
print(Y)

[0 0 0 ... 1 1 1]


In [25]:
#SPLITTING OF DATA
X_train, X_test, Y_train, Y_test =train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [26]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [27]:
print(X_test)

['mmangen fine havent much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini41 nevertheless hooray 4700 member wonder safe trip' 'feel well'
 'supersandro thank']


In [28]:
print(X_test)

['mmangen fine havent much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini41 nevertheless hooray 4700 member wonder safe trip' 'feel well'
 'supersandro thank']


In [29]:
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [30]:
print(X_train)

  (0, 586697)	0.44920535351611973
  (0, 310033)	0.4200808718988117
  (0, 154288)	0.3749490854173817
  (0, 252031)	0.5288703526392218
  (0, 470313)	0.3571791025044959
  (0, 577558)	0.27180697859916886
  (1, 242771)	0.2283623047697854
  (1, 220106)	0.9735762208272303
  (2, 377876)	0.16860156871902512
  (2, 183316)	0.20354126077367451
  (2, 206407)	0.18889886338142908
  (2, 541212)	0.15310110203078062
  (2, 350160)	0.24375172482875457
  (2, 587220)	0.336807788954361
  (2, 113932)	0.31557120499664315
  (2, 573582)	0.33214379168378816
  (2, 537187)	0.32248714311482524
  (2, 179779)	0.2915565228573618
  (2, 538562)	0.1882972742938162
  (2, 172486)	0.19047281532070873
  (2, 242771)	0.1180439288381628
  (2, 154288)	0.4621573052658722
  (3, 236597)	0.37428704757163533
  (3, 544506)	0.2721883817840884
  (3, 515153)	0.3945984480715155
  :	:
  (1279996, 574147)	0.2683114244854865
  (1279996, 416766)	0.20993274517728436
  (1279996, 313580)	0.22159678677455302
  (1279996, 380988)	0.1791747151413057


In [31]:
print(X_test)

  (0, 558232)	0.1757692534482421
  (0, 541212)	0.3072074651776771
  (0, 527910)	0.3386343192670074
  (0, 514723)	0.21463522268104118
  (0, 366447)	0.17346316523384325
  (0, 355492)	0.44012114207980857
  (0, 235147)	0.27404979666958745
  (0, 220436)	0.2180472992006389
  (0, 190895)	0.23194502054309887
  (0, 183387)	0.24911907064488553
  (0, 149947)	0.3587178274810712
  (0, 101347)	0.2612527558121805
  (0, 54650)	0.15826878878977094
  (0, 33489)	0.1723483714683162
  (1, 485252)	0.24293304041529956
  (1, 462335)	0.4807822485858285
  (1, 337575)	0.28409748242225535
  (1, 289135)	0.4072935197047041
  (1, 200003)	0.5694284898803428
  (1, 33489)	0.2143131631278303
  (1, 22419)	0.3050469465999609
  (2, 529990)	0.43400944740371583
  (2, 337658)	0.2578434736828828
  (2, 249360)	0.5864253166368479
  (2, 129157)	0.3631191019181466
  :	:
  (319995, 558232)	0.25519936465252313
  (319995, 552096)	0.2664155846404765
  (319995, 424925)	0.412078644467028
  (319995, 413397)	0.3194387363688261
  (319995, 

In [32]:
model=LogisticRegression()
model.fit(X_train,Y_train)
Y_pred=model.predict(X_train)
print(accuracy_score(Y_train,Y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.81376015625


In [33]:
Y_test_pred=model.predict(X_test)
print(accuracy_score(Y_test,Y_test_pred))

0.7823


In [34]:
# Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()

# Fit the model
nb_model.fit(X_train, Y_train)

MultinomialNB()

In [35]:
# Predict on the training and testing data
Y_train_pred_nb = nb_model.predict(X_train)
Y_test_pred_nb = nb_model.predict(X_test)

# Calculate the accuracy scores
train_accuracy_nb = accuracy_score(Y_train, Y_train_pred_nb)
test_accuracy_nb = accuracy_score(Y_test, Y_test_pred_nb)

print("Naive Bayes - Training Accuracy:", train_accuracy_nb)
print("Naive Bayes - Testing Accuracy:", test_accuracy_nb)

Naive Bayes - Training Accuracy: 0.8367171875
Naive Bayes - Testing Accuracy: 0.75928125


SAVING THE TRAINED MODEL

In [36]:
import pickle
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [37]:
loaded_model=pickle.load(open(filename,'rb'))

In [38]:
X_new=X_test[200]
prediction=loaded_model.predict(X_new)
print("Y_test value is ",Y_test[200])
print("target value predicted by the saved model is ",prediction)
if(prediction[0]==0):
  print("Positive tweet")
else:
  print("Negative tweet")



Y_test value is  1
target value predicted by the saved model is  [1]
Negative tweet
